In [1]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score
from lightgbm.sklearn import LGBMRegressor
from sklearn.model_selection import KFold,StratifiedKFold
import time
from tqdm.notebook import tqdm
import os

pd.set_option('display.max_rows',800)
pd.set_option('display.max_columns',400)

In [2]:
df_train = pd.read_csv(r'data/环境空气质量评价挑战赛_复赛数据集/train.csv')
df_test = pd.read_csv(r'data/环境空气质量评价挑战赛_复赛数据集/test.csv')

result_path = './sub'
if not os.path.exists(result_path): # 如果不存在则创建目录
    os.makedirs(result_path)

df = pd.concat([df_train, df_test]).reset_index(drop=True)

# 其他特征
df['PM'] = df['PM2_5']/df['PM10']
for col in ['PM2_5','PM10']:
    df[f'AQI/{col}'] = df['AQI']/df[col]

## 特征工程

### IAQI计算

In [3]:
def IAQI_cal(x,xbins):
    if x<xbins[1]:
        IAQI = (x-xbins[0])/(xbins[1]-xbins[0])*50
    elif xbins[1]<=x<xbins[2]:
        IAQI = (x-xbins[1])/(xbins[2]-xbins[1])*50 + 50
    elif xbins[2]<=x<xbins[3]:
        IAQI = (x-xbins[2])/(xbins[3]-xbins[2])*50 + 100
    elif xbins[3]<=x<xbins[4]:
        IAQI = (x-xbins[3])/(xbins[4]-xbins[3])*50 + 150
    elif xbins[4]<=x<xbins[5]:
        IAQI = (x-xbins[4])/(xbins[5]-xbins[4])*100 + 200
    elif xbins[5]<=x<xbins[6]:
        IAQI = (x-xbins[5])/(xbins[6]-xbins[5])*100 + 300
    elif xbins[6]<=x<xbins[7]:
        IAQI = (x-xbins[6])/(xbins[7]-xbins[6])*100 + 400
    elif x>=xbins[7]:
        IAQI = 500
    else:
        print(x,'error')
        IAQI=99999
    IAQI = np.ceil(IAQI)
    return IAQI


PM2 = [0,35,75,115,150,250,350,500]
PM10 = [0,50,150,250,350,420,500,600]
SO2 = [0,50,150,475,800,1600,2100,2620]
CO = [0,2,4,14,24,36,48,60]
NO2 = [0,40,80,180,280,565,750,940]
O3_8h = [0,100,160,215,265,800,10000,10000]
col_bin = ['PM2', 'PM10', 'SO2', 'CO', 'NO2', 'O3_8h']
col_IAQI = []

# 计算每个指标的IAQI值
for col,xbin in zip(['PM2_5', 'PM10', 'SO2', 'CO', 'NO2', 'O3'],col_bin):
    df[f'IAQI_{col}'] = df[col].apply(lambda x: IAQI_cal(x,eval(xbin)))#
    col_IAQI.append(f'IAQI_{col}')

df['IAQI_sum'] = df[col_IAQI].sum(axis=1)
df['IAQI'] = df[col_IAQI].max(axis=1)

### 质量等级特征分解

In [4]:
# 优0-50 
df.loc[(df['IAQI']<=40),'质量等级'] = 1
df.loc[(40<df['IAQI'])&(df['IAQI']<=45),'质量等级'] = 2
df.loc[(45<df['IAQI'])&(df['IAQI']<=50),'质量等级'] = 3

# 良51-100 
df.loc[(50<df['IAQI'])&(df['IAQI']<=55),'质量等级'] = 4
df.loc[(55<df['IAQI'])&(df['IAQI']<=65),'质量等级'] = 5
df.loc[(55<df['IAQI'])&(df['IAQI']<=65),'质量等级'] = 5
df.loc[(65<df['IAQI'])&(df['IAQI']<=74),'质量等级'] = 6
df.loc[(74<df['IAQI'])&(df['IAQI']<=84),'质量等级'] = 7
df.loc[(84<df['IAQI'])&(df['IAQI']<=94),'质量等级'] = 8
df.loc[(94<df['IAQI'])&(df['IAQI']<=100),'质量等级'] = 9

# 轻度污染100-150
df.loc[(100<df['IAQI'])&(df['IAQI']<=110),'质量等级'] = 10
df.loc[(110<df['IAQI'])&(df['IAQI']<=125),'质量等级'] = 11
df.loc[(125<df['IAQI'])&(df['IAQI']<=150),'质量等级'] = 12

# 中度污染151-200
df.loc[(150<df['IAQI'])&(df['IAQI']<=175),'质量等级'] = 13
df.loc[(175<df['IAQI'])&(df['IAQI']<=200),'质量等级'] = 14

# 重度污染201-300
df.loc[(200<df['IAQI'])&(df['IAQI']<=230),'质量等级'] = 15
df.loc[(230<df['IAQI'])&(df['IAQI']<=350),'质量等级'] = 16

# 严重污染301-500  
#t1 350 t2 400 0.00630
#t1 340 t2 400 0.00624 
# 400-500 等级1.03 0.00611
df.loc[(300<df['IAQI'])&(df['IAQI']<=340),'质量等级'] = 17
# df.loc[(340<df['IAQI'])&(df['IAQI']<=400),'质量等级'] = 18
# df.loc[(400<df['IAQI'])&(df['IAQI']<=500),'质量等级'] = 19

Levels = [1.387, 1.465, 1.597, 1.233, 1.34, 1.256, 1.264, 1.407, 1.393, 1.171, 
          1.187, 1.32, 1.13, 1.206, 1.046, 1.125, 1.02, 1.01, 1.03]#1.06, 1.064 1.008
len(Levels)
df['Levels'] = df['质量等级'].apply(lambda x:Levels[int(x-1)])

# label变换#np.log()
w1 = 133
df['new_label'] = df['IPRC'] - df['IAQI'] / w1
df['IAQI_sum/AQI'] = df['IAQI_sum']/np.power(df['IAQI'],df['Levels'])

### IAQI-LR

In [5]:
all_info = {}
for kind, kind_data in df.groupby("质量等级"):
    if kind<17:
        info = {}
        # 线性拟合
        x = kind_data[kind_data['new_label'].notnull()]['IAQI_sum/AQI'].values.reshape(-1, 1)
        y = kind_data[kind_data['new_label'].notnull()]['new_label']
        model = LinearRegression()
        bb = model.fit(x,y)


        info['w'] = model.coef_[0] 
        info['b'] = model.intercept_
        all_info[kind] = info
    else:
        info = {}
        info['w'] = 0.62#0.62
        info['b'] = -1.506#-1.5
        all_info[kind] = info
temp = pd.DataFrame(all_info).T.reset_index().rename(columns={"index": "质量等级"})
df = df.merge(temp, how='left', on='质量等级')

In [6]:
df['feat_LR'] = df['w']*df['IAQI_sum/AQI'] + df['b']
# IAQI排名
df['AQI2'] = df[col_IAQI].apply(lambda x: x.sort_values()[-2],axis=1)/np.power(df['IAQI'],df['Levels'])
df['AQI3'] = df[col_IAQI].apply(lambda x: x.sort_values()[-3],axis=1)/np.power(df['IAQI'],df['Levels'])
df['AQI4'] = df[col_IAQI].apply(lambda x: x.sort_values()[-4],axis=1)/np.power(df['IAQI'],df['Levels'])
df['AQI5'] = df[col_IAQI].apply(lambda x: x.sort_values()[-5],axis=1)/np.power(df['IAQI'],df['Levels'])
df['AQI6'] = df[col_IAQI].apply(lambda x: x.sort_values()[-6],axis=1)/np.power(df['IAQI'],df['Levels'])
# df['IAQI12/AQI'] = np.log((df['AQI']+df['AQI2'])/df['AQI'])

### 特征筛选

In [7]:
df['new_label2'] = df['new_label'] - df['feat_LR']
# df = df.drop(['PM2.5','PM10'],axis=1)
df_train = df.loc[:len(df_train)-1].reset_index(drop=True)
df_test = df.loc[len(df_train):].reset_index(drop=True)
print(df_train.shape, df_test.shape)

feats = [x for x in df.columns if x not in ['date', 'IPRC', 'new_label',
                                            'PM2.5','PM10','SO2', 'CO', 'NO2', 'O3_8h', 
                                            'IAQI_PM2.5', 'IAQI_PM10','IAQI_SO2', 'IAQI_CO', 'IAQI_NO2', 'IAQI_O3_8h',
                                           'IAQI_sum_min','IAQI_sum','IAQI_sum/AQI','AQI_new', 'IAQI_std',
                                           'w', 'b','new_label2','质量等级']]#
# ,'AQI2'
print(feats)
df.head()

(490, 33) (211, 33)
['AQI', 'PM2_5', 'O3', 'PM', 'AQI/PM2_5', 'AQI/PM10', 'IAQI_PM2_5', 'IAQI_O3', 'IAQI', 'Levels', 'feat_LR', 'AQI2', 'AQI3', 'AQI4', 'AQI5', 'AQI6']


,date,AQI,IPRC,PM2_5,PM10,SO2,NO2,CO,O3,PM,AQI/PM2_5,AQI/PM10,IAQI_PM2_5,IAQI_PM10,IAQI_SO2,IAQI_CO,IAQI_NO2,IAQI_O3,IAQI_sum,IAQI,质量等级,Levels,new_label,IAQI_sum/AQI,w,b,feat_LR,AQI2,AQI3,AQI4,AQI5,AQI6,new_label2
0,2014/6/1,78.0,0.480805,54.375000,102.875000,13.625000,21.000000,1.672917,75.166667,0.528554,1.434483,0.758202,75.0,77.0,14.0,42.0,27.0,38.0,273.0,77.0,7.0,1.264,-0.098143,1.126259,0.231910,-0.359634,-0.098443,0.309412,0.173271,0.156769,0.111388,0.057757,0.000300
1,2014/6/2,101.0,0.587508,71.333333,140.250000,12.958333,23.875000,1.959167,82.833333,0.508616,1.415888,0.720143,96.0,96.0,13.0,49.0,30.0,42.0,326.0,96.0,9.0,1.393,-0.134297,0.564825,0.542425,-0.441205,-0.134830,0.166329,0.084897,0.072769,0.051978,0.022524,0.000533
2,2014/6/3,86.0,0.525599,58.458333,120.208333,13.750000,17.458333,1.764167,91.000000,0.486308,1.471133,0.715425,80.0,86.0,14.0,45.0,22.0,46.0,293.0,86.0,8.0,1.407,-0.121017,0.555940,0.513526,-0.405832,-0.120341,0.151793,0.087281,0.085383,0.041743,0.026564,-0.000676
3,2014/6/4,59.0,0.364717,33.958333,65.541667,10.083333,17.791667,1.666667,50.541667,0.518118,1.737423,0.900191,49.0,58.0,11.0,42.0,23.0,26.0,209.0,58.0,5.0,1.340,-0.071373,0.906051,0.232283,-0.282104,-0.071644,0.212424,0.182077,0.112715,0.099709,0.047687,0.000271
4,2014/6/5,100.0,0.584368,73.250000,122.666667,12.625000,33.416667,1.769583,60.625000,0.597147,1.365188,0.815217,98.0,87.0,13.0,45.0,42.0,31.0,316.0,98.0,9.0,1.393,-0.152474,0.531997,0.542425,-0.441205,-0.152637,0.146468,0.075759,0.070708,0.052190,0.021886,0.000163


## model

In [8]:
def build_model_lgb(trn_x, trn_y, val_x, val_y):
    model = LGBMRegressor(learning_rate=0.701398375, #0.10361,
                            boosting_type='gbdt',
                            n_estimators=10000,
                            objective='mse',
                            subsample=0.7, 
                            colsample_bytree=0.5,
                            num_leaves=352,#82,
                            reg_lambda=4,#0.6,
                            n_jobs=-1,
                          random_state=2020)
    
    model.fit(trn_x, trn_y,
                eval_set=[(val_x, val_y)],
                eval_metric='mse',
                early_stopping_rounds=1000,
                verbose=False)
    
    # model.best_ntree_limit在early_stopping_rounds中生成
    trn_pred  = model.predict(trn_x)#.reshape(-1,1) 
    val_pred  = model.predict(val_x)#.reshape(-1,1) 
    
    return trn_pred, val_pred, model

Val_rmses = []
Trn_rmses = []
R2s = []
pred_y = 0
importance = 0
feat_importance = pd.DataFrame()

fold_num = 5
skf = KFold(n_splits=fold_num, shuffle=True, random_state=2021)

for i, (trn_idx, val_idx) in tqdm(enumerate(skf.split(df_train[feats], df_train['new_label2']))):
    trn_x, trn_y = df_train.loc[trn_idx,feats].reset_index(drop=True), df_train.loc[trn_idx,'new_label2'].reset_index(drop=True)
    val_x, val_y = df_train.loc[val_idx,feats].reset_index(drop=True), df_train.loc[val_idx,'new_label2'].reset_index(drop=True)
    trn_pred, val_pred, model= build_model_lgb(trn_x, trn_y, val_x, val_y)
    
    # 真实标签
    trn_pred_label = trn_pred+df_train.loc[trn_idx,'feat_LR'] + df_train.loc[trn_idx,'IAQI'] / w1
    val_pred_label = val_pred+df_train.loc[val_idx,'feat_LR'] + df_train.loc[val_idx,'IAQI'] / w1
    Trn_rmse = np.power(mean_squared_error(df_train.loc[trn_idx,'IPRC'], trn_pred_label), 0.5)
    Val_rmse = np.power(mean_squared_error(df_train.loc[val_idx,'IPRC'], val_pred_label), 0.5)
    Val_rmses.append(Val_rmse)
    Trn_rmses.append(Trn_rmse)
    
    # 重要度
    importance += model.booster_.feature_importance(importance_type='gain')/5

    
    pred_y += model.predict(df_test[feats])/fold_num

Trn_rmse_mean = np.mean(Trn_rmses)
Val_rmse_mean = np.mean(Val_rmses)
# print('RMSE:',np.round(Trn_rmses,fold_num),'\n',np.round(Val_rmses,fold_num))
print('Trn_rmse: %.5f'% Trn_rmse_mean,' Val_mse: %.5f' % Val_rmse_mean)

0it [00:00, ?it/s]

Trn_rmse: 0.00160  Val_mse: 0.00177


In [9]:
df_test['IPRC'] = pred_y+df_test['feat_LR'] + df_test['IAQI'] / w1
df_test['AQI'] = df_test['IAQI']
sub = df_test[['date', 'IPRC','AQI']]
sub.to_csv(f'./sub/IPRC_lgb_Trn{int(Trn_rmse_mean*10e4)}_Val{int(Val_rmse_mean*10e4)}.csv'
           ,index=False)